In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_004b import *
import torchvision.models as tvm

# Dogs and cats

## Basic data aug

In [ ]:
PATH = Path('data/dogscats')
arch = tvm.resnet34

In [ ]:
#export
def uniform_int(low, high, size=None):
    return random.randint(low,high) if size is None else torch.randint(low,high,size)

@TfmPixel
def dihedral(x, k:partial(uniform_int,0,8)):
    flips=[]
    if k&1: flips.append(1)
    if k&2: flips.append(2)
    if flips: x = torch.flip(x,flips)
    if k&4: x = x.transpose(1,2)
    return x.contiguous()

In [ ]:
#export
def get_transforms(do_flip=True, flip_vert=False, max_rotate=10., max_zoom=1.1, max_lighting=0.2, max_warp=0.2,
                   p_affine=0.75, p_lighting=0.5, xtra_tfms=None):
    res = [rand_crop()]
    if do_flip:    res.append(dihedral() if flip_vert else flip_lr(p=0.5))
    if max_warp:   res.append(symmetric_warp(magnitude=(-max_warp,max_warp), p=p_affine))
    if max_rotate: res.append(rotate(degrees=(-max_rotate,max_rotate), p=p_affine))
    if max_zoom>1: res.append(rand_zoom(scale=(1.,max_zoom), p=p_affine))
    if max_lighting:
        res.append(brightness(change=(0.5*(1-max_lighting), 0.5*(1+max_lighting)), p=p_lighting))
        res.append(contrast(scale=(1-max_lighting, 1/(1-max_lighting)), p=p_lighting))
    #       train                   , valid
    return (res + listify(xtra_tfms), [crop_pad()])  

imagenet_stats = tensor([0.485, 0.456, 0.406]), tensor([0.229, 0.224, 0.225])

In [ ]:
data_norm,data_denorm = normalize_funcs(*imagenet_stats)

In [ ]:
size=224

tfms = get_transforms(do_flip=True, max_rotate=10, max_zoom=1.2, max_lighting=0.3, max_warp=0.15)
data = data_from_imagefolder(PATH, test='test1', bs=64, ds_tfms=tfms,
                             num_workers=8, tfms=data_norm, size=size)

In [ ]:
(x,y) = next(iter(data.valid_dl))

_,axs = plt.subplots(4,4,figsize=(12,12))
for i,ax in enumerate(axs.flatten()): show_image(data_denorm(x[i].cpu()), ax)

In [ ]:
(x,y) = next(iter(data.test_dl))

_,axs = plt.subplots(4,4,figsize=(12,12))
for i,ax in enumerate(axs.flatten()): show_image(data_denorm(x[i].cpu()), ax)

In [ ]:
x=data.valid_ds[2][0]
_,axes = plt.subplots(2,4, figsize=(12,6))
for i,ax in enumerate(axes.flat): dihedral(x,i).show(ax)

## ConvLearner

In [ ]:
#export
def train_epoch(model, dl, opt, loss_func):
    "Simple training of `model` for 1 epoch of `dl` using optim `opt` and loss function `loss_func`"
    model.train()
    for xb,yb in dl:
        loss = loss_func(model(xb), yb)
        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or 1
        self.ap,self.mp = nn.AdaptiveAvgPool2d(sz), nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

def create_body(model, cut=None, body_fn=None):
    return (nn.Sequential(*list(model.children())[:-cut]) if cut
            else body_fn(model) if body_fn else model)

def num_features(m):
    for l in reversed(flatten_model(m)):
        if hasattr(l, 'num_features'): return l.num_features

In [ ]:
model = create_body(arch(), 2)
num_features(model)

In [ ]:
#export
def bn_drop_lin(n_in, n_out, bn=True, p=0., actn=None):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

def create_head(nf, nc, lin_ftrs=None, ps=0.2):
    lin_ftrs = [nf, 512, nc] if lin_ftrs is None else [nf] + lin_ftrs + [nc]
    ps = listify(ps)
    if len(ps)==1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = [AdaptiveConcatPool2d(), Flatten()]
    for ni,no,p,actn in zip(lin_ftrs[:-1],lin_ftrs[1:],ps,actns): 
        layers += bn_drop_lin(ni,no,True,p,actn)
    return nn.Sequential(*layers)

In [ ]:
create_head(512, 2)

In [ ]:
#export
def cond_init(m, init_fn):
    if (not isinstance(m, bn_types)) and requires_grad(m):
        if hasattr(m, 'weight'): init_fn(m.weight)
        if hasattr(m, 'bias') and hasattr(m.bias, 'data'): m.bias.data.fill_(0.)

def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module): f(m)
    for l in c: apply_leaf(l,f)

def apply_init(m, init_fn): apply_leaf(m, partial(cond_init, init_fn=init_fn))

def _init(learn, init): apply_init(learn.model, init)
Learner.init = _init

class ConvLearner(Learner):
    def __init__(self, data, arch, cut, pretrained=True, lin_ftrs=None, ps=0.5, custom_head=None, **kwargs):
        body = create_body(arch(pretrained), cut)
        nf = num_features(body) * 2
        head = custom_head or create_head(nf, data.c, lin_ftrs, ps)
        model = nn.Sequential(body, head)
        super().__init__(data, model, **kwargs)
        self.split([model[1]])
        if pretrained: self.freeze()
        apply_init(model[1], nn.init.kaiming_normal_)

In [ ]:
lr = 3e-3

## Train

In [ ]:
learn = ConvLearner(data, arch, 2, wd=1e-2, metrics=accuracy)

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('0')

## Unfreeze

In [ ]:
learn.load('0')

In [ ]:
learn.unfreeze()

In [ ]:
lr=3e-4

In [ ]:
learn.fit_one_cycle(6, slice(lr/25,lr), pct_start=0.05)

In [ ]:
learn.fit_one_cycle(6, slice(lr/25,lr), pct_start=0.05)

In [ ]:
learn.save('1')

## TTA

In [ ]:
learn.load('1')

In [ ]:
model = learn.model

In [ ]:
#export
def pred_batch(learn, is_valid=True):
    x,y = next(iter(learn.data.valid_dl if is_valid else learn.data.train_dl))
    return x,learn.model(x).detach()
Learner.pred_batch = pred_batch

def get_preds(model, dl, pbar=None):
    return [torch.cat(o).cpu() for o in validate(model, dl, pbar=pbar)]

def _learn_get_preds(learn, is_test=False):
    return get_preds(learn.model, learn.data.holdout(is_test))
Learner.get_preds = _learn_get_preds

def show_image_batch(dl, classes, rows=None, figsize=(12,15), denorm=None):
    x,y = next(iter(dl))
    if rows is None: rows = int(math.sqrt(len(x)))
    x = x[:rows*rows].cpu()
    if denorm: x = denorm(x)
    show_images(x,y[:rows*rows].cpu(),rows, classes)

In [ ]:
#export
def _tta_only(learn, is_test=False, scale=1.25):
    dl = learn.data.holdout(is_test)
    ds = dl.dataset
    old = ds.tfms
    augm_tfm = [o for o in learn.data.train_ds.tfms if o.tfm not in
               (crop_pad, flip_lr, dihedral, zoom)]
    try:
        pbar = master_bar(range(8))
        for i in pbar:
            row = 1 if i&1 else 0
            col = 1 if i&2 else 0
            flip = i&4
            d = {'row_pct':row, 'col_pct':col, 'is_random':False}
            tfm = [*augm_tfm, zoom(scale=scale, **d), crop_pad(**d)]
            if flip: tfm.append(flip_lr(p=1.))
            ds.tfms = tfm
            yield get_preds(learn.model, dl, pbar=pbar)[0]
    finally: ds.tfms = old
        
Learner.tta_only = _tta_only

In [ ]:
t = list(learn.tta_only(scale=1.35))

In [ ]:
preds,y = get_preds(model, data.valid_dl)

In [ ]:
accuracy(preds, y)

In [ ]:
avg_preds = torch.stack(t).mean(0)
avg_preds.shape, accuracy(avg_preds, y)

In [ ]:
accuracy(preds*0.5 + avg_preds*0.5, y)

In [ ]:
[(beta,accuracy(preds*beta + avg_preds*(1-beta), y)) for beta in np.linspace(0,1,11)]

In [ ]:
#export
def _TTA(learn, beta=0.5, scale=1.35, is_test=False):
    preds,y = learn.get_preds(is_test)
    all_preds = list(learn.tta_only(scale=scale, is_test=is_test))
    avg_preds = torch.stack(all_preds).mean(0)
    return preds*beta + avg_preds*(1-beta), y

Learner.TTA = _TTA

In [ ]:
learn = ConvLearner(data, arch, 2, metrics=accuracy, path=PATH)

In [ ]:
learn.load('0')

In [ ]:
tta_preds = learn.TTA()

In [ ]:
accuracy(tta_preds[0], y)

## Fin